# VCP SDK のセットアップ

この Notebook では VCP SDK を利用するための初期設定と、アプリケーション・テンプレート等のインストールを行います。

## 設定ファイルの準備

VCP SDK を利用する上で必要となる設定ファイルを作成します。

### `vcp_config.yml` ファイルの編集

以下のセルを実行すると、VCP SDK 設定ファイル `vcp_config.yml` を編集するためのリンクボタンが表示されます。

```
vcc:
    host: VCコントローラのIPアドレス (VCPポータブル版を同一ホストに構築した場合は localhost を指定)
    name: VCコントローラの名前 (VCコントローラ上のログ出力に使用)
```

In [ ]:
import os

config_dir = "{}/vcpsdk/vcpsdk/config".format(os.environ.get("HOME"))
!mkdir -p vcp_config
!cp $config_dir/vcp_flavor.yml vcp_config/
!cp $config_dir/sample-vault-vcp_config.yml vcp_config/vcp_config.yml

curdir = os.getcwd().replace('/notebooks/notebook/', '')
config_url = "/{}/edit/vcp_config/vcp_config.yml".format(os.environ['SUBDIR'])
from IPython.core.display import HTML

style= "text-decoration: none;\
padding: 4px 8px;border: solid 2px #ddf; color: #fff;\
font-size: 20px;\
font-weight: bold;\
background-color: rgb(100, 149, 237);"
HTML('<a style="{}" target="new" href="{}">vcp_config.ymlを編集</a>'.
     format(style, config_url))

## クラウド認証情報の書き込み用 Notebook の起動

利用先クラウドプロバイダーの認証情報を Vault サーバのストレージに書き込む作業を簡便に行うための Notebook を起動します。

以下のセルを実行すると、Notebook を開くボタンが表示されます。

In [ ]:
!cp -r $HOME/vcp/vcpsdk/vault/credential_setup.ipynb vcp_config

config_url = "/{}/notebooks/vcp_config/credential_setup.ipynb".format(os.environ['SUBDIR'])
style= "text-decoration: none;\
padding: 4px 8px;border: solid 2px #ddf; color: #fff;\
font-size: 20px;\
font-weight: bold;\
background-color: rgb(100, 149, 237);"
HTML('<a style="{}" target="new" href="{}">クレデンシャル情報の書き込み用Notebookを開く</a>'.
     format(style, config_url))

# ハンズオン・コンテンツの準備

OCS ハンズオンコンテンツのディレクトリとファイルを Jupyter Notebook 環境に展開します。

In [ ]:
!git clone https://github.com/nii-gakunin-cloud/handson.git

# VCP SDK マニュアルの準備

VCP SDK マニュアル最新版のディレクトリとファイルを Jupyter Notebook 環境に展開します。

In [ ]:
!cp -r $HOME/vcp/vcpsdk/doc .

# アプリケーション・テンプレートの準備

OCS アプリケーション・テンプレートのディレクトリとファイルを Jupyter Notebook 環境に展開します。

In [ ]:
!git clone https://github.com/nii-gakunin-cloud/ocs-templates.git templates

# VCP SDK テストセットの準備

VCP SDK のテストセット、サンプル実装のディレクトリとファイルを Jupyter Notebook 環境に展開します。

In [ ]:
!cp -r $HOME/vcp/vcpsdk/sdk_test $HOME/vcp/vcpsdk/sample .